# General Metrics

In [ ]:
import os 
import glob
import PIL.Image
from typing import List, Optional, Union
import numpy as np
import torch
from metrics import Metrics, preprocess_image

device = "cpu"
test_image_dir = "example_data/2c21b97ff3dc4fc3b1ef9e4bb0164318"

def load_images_from_dir(image_dir: str):
    all_images = glob.glob(os.path.join(image_dir, "*.png"))
    all_images = [preprocess_image(PIL.Image.open(image)) for image in all_images]
    torch_image_tensor = torch.tensor(np.array(all_images), dtype=torch.float32)
    # (num_frames, channels, height, width)
    torch_image_tensor = torch_image_tensor.permute(0, 3, 1, 2)
    target = torch_image_tensor.to(device)
    return target

In [ ]:
metrics = Metrics(device=device)
target = load_images_from_dir(test_image_dir)

In [ ]:
for i in range(10):
    input = target + torch.randn_like(target) * 0.001
    input = torch.clamp(input, 0, 1).to(device)
    metrics.compute_image(input, target)
metrics.get_total_metrics()

# Car Quality Metrics

In [1]:
import os
import glob
import PIL.Image
import numpy as np
import torch
from car_quality_estimator.car_quality_metric import load_car_quality_score

test_image_dir = "example_data/2c21b97ff3dc4fc3b1ef9e4bb0164318"
# load_car_quality_score(model_dir="../models", device=None, use_combined_embedding_model=False)
car_quality_metric = load_car_quality_score(
    model_dir="./models/", use_combined_embedding_model=True)

In [2]:
all_images = glob.glob(os.path.join(test_image_dir, "*.png"))
all_images = [PIL.Image.open(image).convert("RGB") for image in all_images]
reference_batch = all_images
# reference_batch = [all_images[i:i+4] for i in range(0, len(all_images), 4)]
all_images_np = np.array(all_images) / 255.0
all_images_distorted_np  = all_images_np + np.random.randn(*all_images_np.shape) * 0.0005
all_images_distorted_np = np.clip(all_images_distorted_np, 0, 1)
all_images_distorted_pil = [PIL.Image.fromarray((img * 255).astype(np.uint8)) for img in all_images_distorted_np]
generated_views = all_images_distorted_pil

In [3]:
car_quality_metric.compute_scores_no_reference(generated_views)

Computing quality scores for generated models...


{'avg_quality_score': 0.15858957,
 'avg_entropy': 0.11579005,
 'avg_combined_score': 0.13669631,
 'quality_std': 0.31417063,
 'num_samples': 21}

In [4]:
car_quality_metric.compare_with_reference(
    generated_views, reference_batch, compute_kid=True)

Generating embeddings for generated models...
Generating embeddings for reference models...
Computing quality scores for generated models...
Computing quality scores for reference models...


{'generated_metrics': {'avg_quality_score': 0.15668477,
  'avg_entropy': 0.119968355,
  'avg_combined_score': 0.1348948,
  'quality_std': 0.30946776,
  'num_samples': 21},
 'reference_metrics': {'avg_quality_score': 0.19824842,
  'avg_entropy': 0.07728706,
  'avg_combined_score': 0.17999125,
  'quality_std': 0.36979467,
  'num_samples': 21},
 'quality_gap': 0.041563645,
 'score_distribution_metrics': {'kl_divergence_kde': -0.02978984316058639,
  'jensen_shannon_distance': 0.37441153045043296,
  'wasserstein_distance': 0.04853838673107496},
 'kid_metrics': {'kid_score': 10802.275148809524,
  'n_gen_samples': 21,
  'n_ref_samples': 21}}